## This will be to analyse the EPS data scraped by DataAssembly/EPS/parse_10q_filings.py
### first analysis is to see how many blanks we have (can I do the rest by hand or not)
### second will be to look for outliers - these will be to look for instances where the LLM screwed up. 


In [99]:
# Analysis for news.db (e.g how many articles are there for each ticker)
import sys, os
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, "../..")))
import config
import pandas as pd
import matplotlib.pyplot as plt

In [100]:
df = pd.read_csv("../../"+config.QUARTERLY_EPS_DATA_CSV)
type(df)
df_quarterly = df[df['Form Type'] == '10-Q (Quarterly report)'].copy()


In [101]:
def find_null_eps_10q_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find rows where all EPS columns are null and the form type is '10-Q'.
    Returns a DataFrame with index, Ticker, and AccessionNumber.
    """
    initial_count = len(df)


    mask = (
        (df['quarterly_raw_eps'].isnull()) &
        (df['quarterly_diluted_eps'].isnull()) &
        (df['annual_raw_eps'].isnull()) &
        (df['annual_diluted_eps'].isnull())
    )

    result_df = df.loc[mask, ['Ticker', 'Accession Number']].copy()
    print(f"Total of {initial_count} rows.")
    print(f"[INFO] Found {len(result_df)} rows with all EPS values missing for 10-Q filings. ({len(result_df)*100/initial_count}%)")
    return result_df


In [102]:
missing_df = find_null_eps_10q_rows(df=df_quarterly)
missing_df.head(10)

Total of 6163 rows.
[INFO] Found 138 rows with all EPS values missing for 10-Q filings. (2.2391692357618043%)


,Ticker,Accession Number
178,TER,0001193125-17-255654
260,WDC,0000106040-24-000040
261,WDC,0000106040-24-000022
262,WDC,0000106040-24-000014
263,WDC,0000106040-23-000034
264,WDC,0000106040-23-000017
265,WDC,0000106040-23-000010
269,WDC,0000106040-21-000053
506,ROST,0001206774-12-002608
799,TSLA,0001564590-18-019254


In [111]:
ticker_missing_counts_df = missing_df['Ticker'].value_counts().reset_index()
ticker_missing_counts_df.columns = ['Ticker', 'MissingCount']
print(len(ticker_missing_counts_df))
print(ticker_missing_counts_df)
# Tickers with no articles:
# [TEL,MCHP,BAC,EG,AJG(7),PTC]
# 22 + 7 + 7 + 3 + 2 + 1 = 42
# 140 - 42 = 98
# I can do this by hand

19
   Ticker  MissingCount
0      BX            23
1     TEL            22
2     KKR            20
3    TSLA            16
4      MS            11
5     WDC             7
6    MCHP             7
7     HAS             6
8     BAC             5
9    ULTA             4
10    AMD             4
11     EG             3
12    PGR             2
13   BKNG             2
14    AJG             2
15   ERIE             1
16   ROST             1
17    PTC             1
18    TER             1


In [104]:
def flag_eps_anomalies(df: pd.DataFrame, threshold_pct: float = 5.0) -> pd.DataFrame:
    """
    Flags EPS rows where:
    - Raw and diluted EPS have opposite signs
    - Absolute percentage difference exceeds threshold AND absolute difference > 0.05
    - Absolute raw EPS < absolute diluted EPS
    """

    # Filter for rows with both EPS values present
    mask = df['quarterly_raw_eps'].notnull() & df['quarterly_diluted_eps'].notnull()
    eps_df = df.loc[mask].copy()

    raw_eps = eps_df['quarterly_raw_eps']
    diluted_eps = eps_df['quarterly_diluted_eps']

    abs_raw = raw_eps.abs()
    abs_diluted = diluted_eps.abs()

    # Absolute difference
    abs_diff = (raw_eps - diluted_eps).abs()

    # Percentage difference
    eps_df['percentage_difference'] = ((abs_raw - abs_diluted) / abs_raw.replace(0, float('nan'))) * 100

    # Conditions
    sign_mismatch = (raw_eps * diluted_eps) < 0
    pct_difference = (eps_df['percentage_difference'].abs() > threshold_pct) & (abs_diff > 0.02)
    raw_smaller = abs_raw < abs_diluted

    # Combined condition
    anomaly_mask = sign_mismatch | pct_difference | raw_smaller

    return eps_df.loc[anomaly_mask, [
        'Ticker', 'Query', 'quarterly_raw_eps', 'quarterly_diluted_eps', 'percentage_difference'
    ]]


In [110]:
anomalies = flag_eps_anomalies(df_quarterly, threshold_pct=20)
print(anomalies.head(50))
print(f"Total anomalies found: {len(anomalies)}")

     Ticker                      Query  quarterly_raw_eps  \
815    TSLA  TSLA/0001193125-13-212354               0.10   
889    WYNN  WYNN/0001174922-23-000105               0.11   
1736   VRSN  VRSN/0001014473-18-000028               1.38   
5856     GM    GM/0001467858-14-000125               0.08   

      quarterly_diluted_eps  percentage_difference  
815                    0.00             100.000000  
889                   -0.02              81.818182  
1736                   1.09              21.014493  
5856                   0.06              25.000000  
Total anomalies found: 4


In [106]:
from parse_10q_filings import extract_eps

def reprocess_anomalies(df: pd.DataFrame, anomaly_df: pd.DataFrame, output_path: str = None) -> pd.DataFrame:
    """
    Reprocess EPS values for flagged anomaly rows and update the original DataFrame.

    Args:
        df (pd.DataFrame): Original full DataFrame with EPS data.
        anomaly_df (pd.DataFrame): DataFrame of flagged anomalies, must include 'Query', 'Ticker', 'Accession Number'.
        output_path (str, optional): If provided, saves the updated DataFrame to this path.

    Returns:
        pd.DataFrame: Updated copy of the original DataFrame.
    """
    df_copy = df.copy()

    for idx, row in anomaly_df.iterrows():
        query = row['Query']

        # Verify index exists
        if idx not in df_copy.index:
            print(f"[WARN] Index {idx} not found in original DataFrame. Skipping.")
            continue

        # Verify query matches at index
        if df_copy.loc[idx, 'Query'] != query:
            print(f"[WARN] Query mismatch at index {idx}. Skipping to avoid overwrite.")
            continue

        print(f"\n[INFO] Reprocessing index {idx} | Query: {query}")
        print(f"Original EPS values -> Basic: {df_copy.loc[idx, 'quarterly_raw_eps']}, "
              f"Diluted: {df_copy.loc[idx, 'quarterly_diluted_eps']}")

        try:
            new_basic, new_diluted = extract_eps(query)
            print(f"Updated EPS values  -> Basic: {new_basic}, Diluted: {new_diluted}")

            df_copy.at[idx, 'quarterly_raw_eps'] = new_basic
            df_copy.at[idx, 'quarterly_diluted_eps'] = new_diluted

        except Exception as e:
            print(f"[ERROR] Failed to extract EPS for index {idx}: {e}")

    if output_path:
        df_copy.to_csv("../../"+output_path, index=False)
        print(f"[INFO] Updated DataFrame saved to: {output_path}")

    return df_copy


In [107]:
reprocess_anomalies(df=df,
                     anomaly_df=anomalies,
                     output_path=config.EPS_DATA_CSV)


[INFO] Reprocessing index 815 | Query: TSLA/0001193125-13-212354
Original EPS values -> Basic: 0.1, Diluted: 0.0
[INFO] Extracted 2 text blocks, using all for prompt.
[INFO] Attempt 1: Using model llama3-70b-8192
Prompt tokens: 2095, Completion tokens: 16, Total: 2111
[INFO] Received response: basic_eps: 0.10, diluted_eps: 0.00
Updated EPS values  -> Basic: 0.1, Diluted: 0.0

[INFO] Reprocessing index 889 | Query: WYNN/0001174922-23-000105
Original EPS values -> Basic: 0.11, Diluted: -0.02
[INFO] Extracted 3 text blocks, using all for prompt.
[INFO] Attempt 1: Using model llama3-70b-8192
Prompt tokens: 1542, Completion tokens: 16, Total: 1558
[INFO] Received response: basic_eps: 0.11, diluted_eps: -0.02
Updated EPS values  -> Basic: 0.11, Diluted: -0.02

[INFO] Reprocessing index 1736 | Query: VRSN/0001014473-18-000028
Original EPS values -> Basic: 1.38, Diluted: 1.09
[INFO] Extracted 4 text blocks, using all for prompt.
[INFO] Attempt 1: Using model llama3-70b-8192
Prompt tokens: 194

OSError: Cannot save file into a non-existent directory: '../..Data'